In [ ]:
# General modules
import sys
import os
import os.path
from pathlib import Path # creating directories

# math modules
import numpy as np
import tensorflow as tf # used for random seed and setting GPU
## math random seed module
import random as python_random

# load data
from training_modules.load_data import DATA_LOADER
# handling files
from training_modules.handling_saving_stats import get_file_path, save_history, calc_tst_acc, calc_advantage, save_file, display_results

from tensorflow import keras
from keras.models import load_model # for loading a pre-trained key model from file
# ## constants for handling file types
from training_modules.misc import MODEL_CONST, LOSS_CONST, ACC_CONST, ADV_CONST, TRN_GRPH_CONST, ADV_GRPH_CONST, TST_ACC_CONST, VAL_LOSS_CONST, VAL_ACC_CONST, VAL_ADV_CONST, TST_ADV_CONST
# ## hyper parameters
from hyper_parameters import my_seeds, tot
# ## constants for DB_TYPE
from training_modules.misc import TYPE_ASCAD, TYPE_NTRU, TYPE_GAUSS, TYPE_DPA, TYPE_M4SC

# training:
from training_modules.train import train_model

In [ ]:
# prepare
models = dict()

# define empty history object
history = dict()

In [ ]:
def read_parameters_from_file(param_filename):
    #read parameters for the train_model and load_traces functions
    #TODO: sanity checks on parameters
    param_file = open(param_filename,"r")

    #TODO: replace eval() by ast.linear_eval()
    my_parameters= eval(param_file.read())

    my_database = my_parameters["database"]
    my_database_title = my_parameters["database_title"]
    return my_database_title, my_database


In [ ]:
## ASCAD: Adapted heavily by Mahmoud Gharra

# NOTE: code could take another database if it has  supported format as the one provided.


if __name__ == "__main__":

    
    if len(sys.argv)!=2:
        # my_database <string>: path to data file
        # DB_TYPE <int>: 
        ## sets type of parsing for DB...
        ### (See available data types in SCA-with-keras/training_modules/misc.py)

        # network_type <string>:
        ## Architecture Type -- ATM you can choose between 'mlp', 'cnn', 'cnn2', and 'cnn3'
        ### (Look at SCA-with-keras/training_modules/training_models.py for a better overview of the available architectures)


        # training_model <string>:
        ## save folder, in which all the training models, graphs and results are saved. It is created once the program starts.
        ## You may choose an existing program to resume a training.

        # DB_title <string>:
        ## Name of Architecture to be used for display in graphs, can be chosen at random. (Cosmetic...)

        # epochs <int>:
        ## Can be anywhere between 20 and 200 depending on the architecture and the other Hyperparameters

        # LEARNING_RATE <float>:
        ## anwhere between 1e-10 and 1e-1...
        ### Your choice depends on your chosen architecture, number of epochs and many other hyper-parameters



        my_database = "../GS.pickle"
        DB_title = "Gaussian Sampler"
        DB_TYPE = TYPE_GAUSS 

        # CNN training
        network_type = "mlp" ## ATM: you can choose between 'mlp', 'cnn', 'cnn2', and 'cnn3'
        # save folder
#         training_model = "../refactored/sca_GS_mlp_lr1e-5_ep50" # save file for results
        training_model = "../refactored/testtesttest1" # save file for results
        
        ###### Commented out examples ########
        # default parameters values
        # my_database = "../chipWhisp01/projects/operand_scanning_32" # Loc on Einstein
        # DB_title = "operand_scanning_32" # arbitrary name
        # DB_TYPE = TYPE_NTRU

        # DB_title = my_database = "../schoolbook32/schoolbook32" # loc on Einstein
        # # DB_title = "schoolbook32" ## Optional... It's for the graph
        # DB_TYPE = TYPE_NTRU

        # my_database = "../PRE_MAY_06/ASCAD/ATMEGA_AES_v1/ATM_AES_v1_fixed_key/ASCAD_data/ASCAD_databases/ASCAD.h5" # Loc on Einstein
        # DB_title = "ASCAD"
        # DB_TYPE = TYPE_ASCAD

        # my_database = "../dpa_data"
        # DB_title = "DPA contest"
        # DB_TYPE = TYPE_DPA

        # my_database = "../2020-07-30-115642-118eafdd-ntruprime.pickle"
        # DB_title = "M4SC"
        # DB_TYPE = TYPE_M4SC

        
        
        epochs = 2
        LEARNING_RATE = 0.00001
        batch_size = 100

    else:
        #get parameters from user input
        my_database, DB_TYPE, network_type, training_model, epochs, LEARNING_RATE, batch_size = read_parameters_from_file(sys.argv[1])
        DB_title = my_database
        
        
    # create save directory, if it doesn't exist
    Path(training_model).mkdir(parents=True, exist_ok=True)


    accuracies = {}
    advantages = {}
    ### training
    print("+ Commense training (takes about 20 seconds per epoch)")

    # with tf.device('/gpu:0'):

    for seed in my_seeds:
        # increasing reproducability of results according to https://keras.io/getting_started/faq/
        # set PYTHONHASHSEED before running program to increase reproducability
        # You can also consider using the CPU, as GPUs are less deterministic. (CUDA="")
        np.random.seed(seed)
        python_random.seed(seed)
    #     tf.random.set_random_seed(seed) # older tf versions use this
        tf.random.set_seed(seed)

        data_loader = DATA_LOADER(DB_TYPE, my_database)

        print("++ Data dimestions are: ", np.array(data_loader.X).shape)

        # sample some traces
        # Sampling traces intereferes with the resulting graph. This must mean that I'm doing that suboptimally
        # TODO: fix it, so that saved graph is correct
        print("+ sample {0} traces".format(tot))
        if tot != 0:
            sample_traces(X)
            print("+ print out list of categories:\n {}".format(np.unique(y)))
            print("+ print out shape of labels:\n {}".format(y.shape))


        for key_idx in range(data_loader.KEY_LENGTH):
            print('key_idx={}...'.format(key_idx))

            # If it hasn't been trained. Load a model, extract the profiling and attack traces and labels and commense the training
            ## extract profiling and testing data and create a model
            data_loader.extract_data(network_type, lr=LEARNING_RATE, key_idx=key_idx)

            # Check if this seed key combination had been trained in the past
            ## we run individual training for each key integer
            if models.get((seed, key_idx)) is not None:
                data_loader.model = load_model(models.get((seed, key_idx)))
                ## calculate ACCURACY
                accuracies[(seed, key_idx)] = calc_tst_acc(data_loader, seed=seed, key_idx=key_idx)
                ## calculate ADVANTAGE
                advantages[(seed, key_idx)] = calc_advantage(accuracies.get((seed, key_idx)))
                continue

            _file_path = os.path.normpath(get_file_path(training_model, MODEL_CONST, seed, key_idx))
            if os.path.exists(_file_path) == True:
                models[(seed, key_idx)] =  get_file_path(training_model, MODEL_CONST, seed, key_idx)
                data_loader.model = load_model(models.get((seed, key_idx)))
                ## calculate ACCURACY
                accuracies[(seed, key_idx)] = calc_tst_acc(data_loader, seed=seed, key_idx=key_idx)
                ## calculate ADVANTAGE
                advantages[(seed, key_idx)] = calc_advantage(accuracies.get((seed, key_idx)))
                continue                


            ## train model, get number of attempts needed and history struct
            history, att = train_model(data_loader, training_model, epochs=epochs, batch_size=batch_size, seed=seed, key_idx=key_idx)
            ## epochs, batch_size, MIN_ACC, drop_out, MAX_ATTEMPTS_PER_KEY, LEARNING_RATE, validation_split_const
            
            
            ## TODO: save hist
            print("+ save history")
            # SAVE HISTORY
            save_history(training_model, history, seed, key_idx, att)

            # add file path to list of trained models
            models[(seed, key_idx)] = get_file_path(training_model, MODEL_CONST, seed, key_idx) # TODO: models can be removed, get_file_path() is better at getting model instance
            # TODO: THE FOLLOWING 4 LINES WERE REPEATED 4 TIMES
            ## calculate test accuracy, you'll be saving them all at the end
            accuracies[(seed, key_idx)] = calc_tst_acc(data_loader, seed=seed, key_idx=key_idx)

            ## calculate ADVANTAGE
            advantages[(seed, key_idx)] = calc_advantage(accuracies.get((seed, key_idx)))


    print()
    print("+ Training complete")

    # Save all test accuracies and advantages
    print("+ Saving dictionary with all test accuracies and test advantages of all seeds (and keys if those exist)")
    save_file(training_model, accuracies, case=TST_ACC_CONST)
    save_file(training_model, advantages, case=TST_ADV_CONST)



    # TODO: replace calculation of advantage with loading of it
    display_results(models, accuracies, advantages, training_model, DB_TYPE = DB_TYPE)




In [ ]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())



In [ ]:
# import tensorflow as tf
# sess = tf.compat.v1.sess(config=tf.ConfigProto(log_device_placement=True))


In [ ]:
# 0.5171052631578947

In [ ]:
# import tensorflow as tf

# # print("Total Devs Available: ", len(tf.config.experimental.list_physical_devices()))
# # print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices("GPU")))
# # print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices("CPU")))
# # import tensorflow as tf
# tf.config.experimental.list_physical_devices()

In [ ]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())


In [ ]:
# import tensorflow as tf
# tf.test.gpu_device_name()


In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# gpus

In [ ]:
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))


In [ ]:
# print(device_lib.list_local_devices())
# tf.Session()
# tf.compat.v1.Session()

In [ ]:
# print(tf.config.list_physical_devices('GPU'))


In [ ]:
# from __future__ import absolute_import, division, print_function, unicode_literals

# import numpy as np
# import tensorflow as tf
# # import tensorflow_hub as hub
# # import tensorflow_datasets as tfds


# tf.config.experimental.list_physical_devices("GPU")

In [ ]:
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# tf.config.experimental.list_physical_devices('GPU') 

In [ ]:
# num = np.int16(800)
# np.array([0 if (num & (1 << i) == 0) else 1 for i in range(15, -1, -1)])